In [1]:
import csv
import heapq

In [2]:
class Node:
    def __init__(self, name, g_cost, h_cost, parent=None):
        self.name = name
        self.g_cost = g_cost  # Cost from start to current node
        self.h_cost = h_cost  # Heuristic cost from current node to goal
        self.f_cost = g_cost + h_cost  # Total cost
        self.parent = parent

    def __lt__(self, other):
        return self.f_cost < other.f_cost

In [3]:
def load_nodes(filename):
    nodes = {}
    with open(filename, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            nodes[row['node_name']] = {
                'type': row['type'],
                'x': float(row['x']),
                'y': float(row['y'])
            }
    return nodes

In [4]:
def load_edges(filename):
    edges = {}
    with open(filename, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            if row['from_node'] not in edges:
                edges[row['from_node']] = []
            edges[row['from_node']].append((row['to_node'], int(row['cost']), row['mode'], int(row['time'])))
    return edges

In [5]:
def load_transfers(filename):
    transfers = {}
    with open(filename, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            if row['node_name'] not in transfers:
                transfers[row['node_name']] = []
            transfers[row['node_name']].append({
                'from_mode': row['from_mode'],
                'to_mode': row['to_mode'],
                'cost': int(row['transfer_cost']),
                'time': float(row['transfer_time'])
            })
    return transfers

In [6]:
def load_dynamic_factors(filename):
    factors = {}
    with open(filename, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            factors[(row['from_node'], row['to_node'])] = {
                'accident_risk': float(row['accident_risk']),
                'congestion_factor': float(row['congestion_factor']),
                'weather_delay': float(row['weather_delay']),
            }
    return factors

In [7]:
def adjust_costs_dynamic(node, to_node, base_cost, current_conditions):
    """
    Adjust the cost dynamically based on conditions like accidents, weather, etc.
    """
    factor = current_conditions.get((node, to_node), {})
    accident_risk = factor.get('accident_risk', 1)
    congestion_factor = factor.get('congestion_factor', 1)
    weather_delay = factor.get('weather_delay', 1)

    w1=0.4 #accident risk
    w2=0.5 #congestion factor
    w3=0.6 #weather delay
    
    adjusted_cost = base_cost * (1 + w1 * accident_risk + w2 * congestion_factor + w3 * weather_delay)

    return adjusted_cost

In [8]:
def manhattan_distance(node, goal_node):
    # Simplified Manhattan distance as heuristic
    return abs(node['x'] - goal_node['x']) + abs(node['y'] - goal_node['y'])


This code works but dont use the dynamic factor to calculate cost

In [24]:
def a_star_search_dynamic(start, goal, nodes, edges, transfers, time_window, dynamic_factors):
    # Initialize open and closed lists
    open_list = []
    closed_list = set()
    
    # Add the start node to the open list
    open_list.append((0, start))
    
    # Dictionary to store the cost of reaching each node
    g_costs = {start: 0}
    
    # Dictionary to store the path
    came_from = {}
    
    while open_list:
        # Get the node with the lowest cost
        current_cost, current_node = min(open_list, key=lambda x: x[0])
        open_list.remove((current_cost, current_node))
        
        # If the goal is reached, reconstruct the path
        if current_node == goal:
            path = []
            while current_node in came_from:
                path.append(current_node)
                current_node = came_from[current_node]
            path.append(start)
            return path[::-1]
        
        closed_list.add(current_node)
        
        # Check if the current node has neighbors in the edges dictionary
        if current_node not in edges:
            continue  # Skip if no neighbors
        
        for neighbor, base_cost, mode, travel_time in edges[current_node]:
            if neighbor in closed_list:
                continue
            
            # Calculate the cost to reach the neighbor
            tentative_g_cost = g_costs[current_node] + base_cost
            
            if neighbor not in g_costs or tentative_g_cost < g_costs[neighbor]:
                g_costs[neighbor] = tentative_g_cost
                total_cost = tentative_g_cost  # Add heuristic if needed
                open_list.append((total_cost, neighbor))
                came_from[neighbor] = current_node
    
    return None  # Return None if no path is found



Code to take care of adjusted factors also

In [28]:
def a_star_search_dynamic(start, goal, nodes, edges, transfers, time_window, current_conditions):
    open_list = []
    closed_list = set()
    
    start_node = Node(start, 0, manhattan_distance(nodes[start], nodes[goal]))
    heapq.heappush(open_list, start_node)
    
    while open_list:
        current_node = heapq.heappop(open_list)
        
        if current_node.name == goal:
            path = []
            while current_node:
                path.append(current_node.name)
                current_node = current_node.parent
            return path[::-1]
        
        closed_list.add(current_node.name)
        
        if current_node.name not in edges:
            continue
        
        for neighbor, base_cost, mode, travel_time in edges[current_node.name]:
            if neighbor in closed_list:
                continue
            
            adjusted_cost = adjust_costs_dynamic(current_node.name, neighbor, base_cost, current_conditions)
            g_cost = current_node.g_cost + adjusted_cost
            
            if g_cost > time_window:
                continue
            
            h_cost = manhattan_distance(nodes[neighbor], nodes[goal])
            neighbor_node = Node(neighbor, g_cost, h_cost, current_node)
            heapq.heappush(open_list, neighbor_node)
    
    return None

In [10]:
# Load data from CSV files
nodes = load_nodes('nodes.csv')

In [34]:
edges = load_edges('edges.csv')

In [12]:
transfers = load_transfers('transfer.csv')

In [32]:
dynamic_factors = load_dynamic_factors('dynamic_factors.csv')

In [22]:
# Define start, goal, and time window
start = 'Delhi'
goal = 'Chennai'
time_window = 100


In [35]:
# Run A* search with dynamic factors
optimal_path = a_star_search_dynamic(start, goal, nodes, edges, transfers, time_window, dynamic_factors)
print("Optimal Path with Dynamic Factors:", optimal_path)

Optimal Path with Dynamic Factors: ['Delhi', 'Kolkata', 'Chennai']
